# УСТАНОВКА И ИМПОРТ

In [3]:
!pip install opencv-python
!pip install torch torchvision
!pip install ultralytics
!pip install tqdm

import os
import cv2
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
from ultralytics import YOLO
from tqdm import tqdm

# СКАЧИВАНИЕ И РАСПАКОВКА АРХИВА ДЛЯ ТЕСТОВ

In [4]:

import urllib.request
import zipfile

dataset_dir = 'MOT17'
os.makedirs(dataset_dir, exist_ok=True)

# Ссылка на датасет
mot17_url = 'https://motchallenge.net/data/MOT17.zip'
zip_path = os.path.join(dataset_dir, 'MOT17.zip')

print("Скачиваем MOT17 (может занять время)...")
urllib.request.urlretrieve(mot17_url, zip_path)
print("Скачивание завершено.")

print("Распаковка...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

os.remove(zip_path)
print("Готово! Датасет распакован в папке:", dataset_dir)


Скачиваем MOT17 (может занять время)...
Скачивание завершено.
Распаковка...
Готово! Датасет распакован в папке: MOT17


# ФУНКЦИИ ЧТЕНИЯ SEQINFO.INI И GT (GROUND TRUTH)

In [5]:
def read_seqinfo(seq_dir):
    """
    Считываем seqinfo.ini в dict:
    {
      'name': ...,
      'imDir': ...,
      'frameRate': ...,
      'seqLength': ...,
      'imWidth': ...,
      'imHeight': ...,
      'imExt': ...
    }
    """
    seqinfo_path = os.path.join(seq_dir, "seqinfo.ini")
    info = {}
    if not os.path.exists(seqinfo_path):
        print("WARNING: seqinfo.ini не найден в", seq_dir)
        return info
    with open(seqinfo_path, 'r') as f:
        for line in f:
            line = line.strip()
            if '=' in line:
                key, val = line.split('=', 1)
                key, val = key.strip(), val.strip()
                info[key] = val
    return info

def read_ground_truth(gt_path):
    """
    Читаем файл GT (gt.txt). Возвращаем dict:
      ground_truth[frame_id] = [(x1,y1,x2,y2,cls=1), ...]
    Учитываем только объекты с class=1 (person).
    """
    annotations = {}

    with open(gt_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            cols = line.split(',')
            # формат: frame, id, x, y, w, h, conf, class, visibility
            frame_id = int(cols[0])
            x = float(cols[2])
            y = float(cols[3])
            w = float(cols[4])
            h = float(cols[5])
            conf = float(cols[6])
            cls  = int(cols[7])  # класс

            # Считаем, что интересует только class=1 => person
            if cls != 1:
                continue

            # Формируем (x1, y1, x2, y2, cls)
            x1, y1 = x, y
            x2, y2 = x + w, y + h

            if frame_id not in annotations:
                annotations[frame_id] = []
            annotations[frame_id].append((x1, y1, x2, y2, cls))
    return annotations


# МЕТРИКИ (IOU, PRECISION, RECALL, F1)

In [6]:
def compute_iou(boxA, boxB):
    """
    boxA, boxB: (x1, y1, x2, y2, ...)
    Возвращаем IoU [0..1].
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = areaA + areaB - interArea
    if unionArea <= 0:
        return 0
    return interArea / unionArea

def match_detections_to_gt(pred_boxes, gt_boxes, iou_thresh=0.5):
    """
    pred_boxes: [(x1, y1, x2, y2, conf, cls), ...]
    gt_boxes:   [(x1, y1, x2, y2, cls), ...]
    Считаем TP, FP, FN при IoU>=iou_thresh.
    """
    matched_gt = set()
    tp = 0
    for pb in pred_boxes:
        p_box = pb[:4]  # (x1,y1,x2,y2)
        found_match = False
        for i, gb in enumerate(gt_boxes):
            if i in matched_gt:
                continue
            g_box = gb[:4]
            iou = compute_iou(p_box, g_box)
            if iou >= iou_thresh:
                tp += 1
                matched_gt.add(i)
                found_match = True
                break
    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - len(matched_gt)
    return tp, fp, fn

def compute_metrics(all_preds, all_gts, iou_thresh=0.5):
    """
    all_preds[frame_id] = [(x1,y1,x2,y2,conf,cls), ...]
    all_gts[frame_id]   = [(x1,y1,x2,y2,cls), ...]
    Возвращаем словарь с Precision, Recall, F1.
    """
    total_tp, total_fp, total_fn = 0, 0, 0
    frame_ids = sorted(set(all_preds.keys()).union(all_gts.keys()))
    for fid in frame_ids:
        preds_f = all_preds.get(fid, [])
        gts_f   = all_gts.get(fid, [])
        tp, fp, fn = match_detections_to_gt(preds_f, gts_f, iou_thresh)
        total_tp += tp
        total_fp += fp
        total_fn += fn

    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp)>0 else 0
    recall    = total_tp / (total_tp + total_fn) if (total_tp + total_fn)>0 else 0
    f1        = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


# ЗАГРУЗКА 3 МОДЕЛЕЙ

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

yolov10_weights = "yolov10n.pt"
yolov11_weights = "yolo11n.pt"

model_y10 = YOLO(yolov10_weights)
model_y11 = YOLO(yolov11_weights)
model_y10.to(device)
model_y11.to(device)

import torchvision.models.detection as models
model_ssd = models.ssd300_vgg16(pretrained=True)
model_ssd.eval()
model_ssd.to(device)

print("Модели загружены.")


Using device: cuda


100%|██████████| 5.59M/5.59M [00:00<00:00, 119MB/s]


100%|██████████| 5.35M/5.35M [00:00<00:00, 124MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:05<00:00, 27.2MB/s]


Модели загружены.


# ФУНКЦИИ ДЛЯ ДЕТЕКЦИИ НА ОДНОМ КАДРЕ

In [8]:
def detect_yolo(model, frame_bgr, device="cpu", conf_thr=0.3, person_class=0):
    """
    Запуск YOLO (v10 или v11) на кадре.
    Возвращаем список [(x1,y1,x2,y2,conf,cls), ...], только для class=person_class.
    """
    results = model(frame_bgr, device=device, verbose=False)
    dets = []
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            conf = box.conf.item()
            cls  = int(box.cls.item())
            if conf >= conf_thr and cls == person_class:
                dets.append((x1, y1, x2, y2, conf, cls))
    return dets

def detect_ssd(model, frame_bgr, device="cpu", conf_thr=0.3, person_class=1):
    """
    Запуск SSD на кадре.
    Возвращаем список [(x1,y1,x2,y2,conf,cls), ...], только для class=person_class (COCO = 1).
    """
    transform = T.Compose([T.ToTensor()])
    inp = transform(frame_bgr).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(inp)[0]

    dets = []
    boxes = out['boxes'].cpu().numpy()
    scores = out['scores'].cpu().numpy()
    labels = out['labels'].cpu().numpy()
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        conf = scores[i]
        cls  = labels[i]
        if conf >= conf_thr and cls == person_class:
            dets.append((x1, y1, x2, y2, conf, cls))
    return dets

# === Настройка классов person для YOLO/SSD ===

yolo_person_cls = 0   # для YOLO
ssd_person_cls  = 1   # для SSD


# ОБРАБОТКА ОДНОЙ ПОСЛЕДОВАТЕЛЬНОСТИ, 3 ОТДЕЛЬНЫХ ВИДЕО

In [9]:
def draw_boxes_on_frame(frame, dets, color=(0,255,0), thickness=2):
    """
    Рисует bounding boxes (x1,y1,x2,y2,conf,cls) на frame (in-place).
    """
    for (x1,y1,x2,y2,cf,cl) in dets:
        cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)
        cv2.putText(frame, f"{cf:.2f}", (int(x1), int(y1)-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

def process_sequence_3videos(seq_dir,
                             model_y10, model_y11, model_ssd,
                             ground_truth,
                             device="cpu",
                             conf_thr=0.3,
                             iou_thr=0.5,
                             person_cls_yolo=0,
                             person_cls_ssd=1):
    """
    1) Читаем seqinfo.ini -> FPS
    2) Читаем кадры из img1/
    3) Для каждого кадра -> запускам 3 модели, рисуем и пишем в 3 отдельные .mp4
    4) Сохраняем предсказания в словарях, по завершении считаем метрики (Precision, Recall, F1)
    5) Возвращаем словари метрик.
    """
    seq_info = read_seqinfo(seq_dir)
    fps = int(seq_info.get('frameRate', 30))

    img_folder = os.path.join(seq_dir, 'img1')
    if not os.path.exists(img_folder):
        print(f"Папка img1 не найдена в {seq_dir}, пропускаем.")
        return None

    frames = sorted([f for f in os.listdir(img_folder) if f.endswith('.jpg')])
    if not frames:
        print(f"Нет .jpg кадров в {img_folder}, пропускаем.")
        return None

    # Берём первый кадр для инициализации VideoWriter
    first_frame_path = os.path.join(img_folder, frames[0])
    first_frame = cv2.imread(first_frame_path)
    if first_frame is None:
        print("Не удаётся прочитать первый кадр, пропускаем.")
        return None
    h, w, _ = first_frame.shape

    # Создаём выходные видеофайлы: YOLOv10, YOLOv11, SSD
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_y10_path = os.path.join(seq_dir, "output_yolov10.mp4")
    out_y11_path = os.path.join(seq_dir, "output_yolov11.mp4")
    out_ssd_path = os.path.join(seq_dir, "output_ssd.mp4")

    vw_y10 = cv2.VideoWriter(out_y10_path, fourcc, fps, (w, h))
    vw_y11 = cv2.VideoWriter(out_y11_path, fourcc, fps, (w, h))
    vw_ssd = cv2.VideoWriter(out_ssd_path, fourcc, fps, (w, h))

    # Словари для метрик
    preds_y10 = {}
    preds_y11 = {}
    preds_ssd = {}

    # Цикл по всем кадрам
    for frame_file in tqdm(frames, desc=f"Обработка {os.path.basename(seq_dir)}"):
        frame_id = int(os.path.splitext(frame_file)[0])  # '000001' -> 1
        frame_path = os.path.join(img_folder, frame_file)
        frame_bgr = cv2.imread(frame_path)
        if frame_bgr is None:
            continue

        # Детекции
        y10_dets = detect_yolo(model_y10, frame_bgr, device=device, conf_thr=conf_thr, person_class=person_cls_yolo)
        y11_dets = detect_yolo(model_y11, frame_bgr, device=device, conf_thr=conf_thr, person_class=person_cls_yolo)
        ssd_dets = detect_ssd(model_ssd, frame_bgr, device=device, conf_thr=conf_thr, person_class=person_cls_ssd)

        # Сохраняем для метрик
        preds_y10[frame_id] = y10_dets
        preds_y11[frame_id] = y11_dets
        preds_ssd[frame_id] = ssd_dets

        # Рисуем (каждую модель на своей копии кадра)
        f_y10 = frame_bgr.copy()
        f_y11 = frame_bgr.copy()
        f_ssd = frame_bgr.copy()

        draw_boxes_on_frame(f_y10, y10_dets, color=(0,255,0))
        draw_boxes_on_frame(f_y11, y11_dets, color=(255,0,0))
        draw_boxes_on_frame(f_ssd, ssd_dets, color=(0,0,255))

        # Допишем текст о модели
        cv2.putText(f_y10, "YOLOv10", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
        cv2.putText(f_y11, "YOLOv11", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 2)
        cv2.putText(f_ssd, "SSD",     (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)

        # Записываем в файлы
        vw_y10.write(f_y10)
        vw_y11.write(f_y11)
        vw_ssd.write(f_ssd)

    # Закрываем видео
    vw_y10.release()
    vw_y11.release()
    vw_ssd.release()

    print(f"\nСохранены три видео:\n  {out_y10_path}\n  {out_y11_path}\n  {out_ssd_path}")

    # Подсчёт метрик
    metrics_y10 = compute_metrics(preds_y10, ground_truth, iou_thr)
    metrics_y11 = compute_metrics(preds_y11, ground_truth, iou_thr)
    metrics_ssd = compute_metrics(preds_ssd, ground_truth, iou_thr)

    print("\n=== МЕТРИКИ ===")
    print("YOLOv10:", metrics_y10)
    print("YOLOv11:", metrics_y11)
    print("SSD:    ", metrics_ssd)

    return metrics_y10, metrics_y11, metrics_ssd


# ЗАПУСК ПО НЕСКОЛЬКИМ ПОСЛЕДОВАТЕЛЬНОСТЯМ MOT

In [10]:
def get_unique_sequences(base_dir):
    """
    Из списка папок выбираем только уникальные последовательности.
    Например: из ['MOT17-01-DPM', 'MOT17-01-FRCNN', 'MOT17-01-SDP']
    оставляем только одну (первую по встречаемости).

    :param base_dir: путь к директории, где хранятся подпапки.
    :return: список уникальных путей к папкам.
    """
    all_subdirs = sorted(os.listdir(base_dir))
    unique_sequences = {}

    for folder in all_subdirs:
        # Разделяем название папки по символу '-'
        parts = folder.split('-')
        if len(parts) < 2:
            continue  # Пропускаем, если имя папки не соответствует формату

        # Основная часть последовательности, например 'MOT17-01'
        base_sequence = '-'.join(parts[:2])

        # Если базовая последовательность ещё не добавлена, добавляем её
        if base_sequence not in unique_sequences:
            unique_sequences[base_sequence] = os.path.join(base_dir, folder)

    return list(unique_sequences.values())

# Основной код запуска
base_dir = "MOT17/MOT17/train"  # train, т.к. там есть GT-файлы (в MOT17 test их нет), чтобы можно было оценить точность

# Выбираем только уникальные последовательности
unique_dirs = get_unique_sequences(base_dir)

for seq_path in unique_dirs:
    seq_name = os.path.basename(seq_path)

    # Проверяем наличие gt/gt.txt
    gt_path = os.path.join(seq_path, 'gt', 'gt.txt')
    if not os.path.exists(gt_path):
        print(f"{seq_name}: нет GT (gt.txt), пропускаем расчёт метрик.")
        continue

    # Читаем GT
    ground_truth = read_ground_truth(gt_path)
    if not ground_truth:
        print(f"{seq_name}: нет объектов class=1 в GT, пропускаем.")
        continue

    print(f"\n=== Обрабатываем последовательность: {seq_name} ===")

    print("Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...")
    metrics_y10, metrics_y11, metrics_ssd = process_sequence_3videos(
        seq_dir=seq_path,
        model_y10=model_y10,
        model_y11=model_y11,
        model_ssd=model_ssd,
        ground_truth=ground_truth,
        device=device,
        conf_thr=0.3,
        iou_thr=0.5,
        person_cls_yolo=yolo_person_cls,
        person_cls_ssd=ssd_person_cls
    )



=== Обрабатываем последовательность: MOT17-02-DPM ===
Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...


Обработка MOT17-02-DPM: 100%|██████████| 600/600 [02:05<00:00,  4.78it/s]



Сохранены три видео:
  MOT17/MOT17/train/MOT17-02-DPM/output_yolov10.mp4
  MOT17/MOT17/train/MOT17-02-DPM/output_yolov11.mp4
  MOT17/MOT17/train/MOT17-02-DPM/output_ssd.mp4

=== МЕТРИКИ ===
YOLOv10: {'precision': 0.8905511811023622, 'recall': 0.18260588773478284, 'f1': 0.303068197043455}
YOLOv11: {'precision': 0.8448857994041709, 'recall': 0.22894354448092138, 'f1': 0.3602642276422764}
SSD:     {'precision': 0.6699612403100775, 'recall': 0.18605026640116248, 'f1': 0.29122614885640874}

=== Обрабатываем последовательность: MOT17-04-DPM ===
Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...


Обработка MOT17-04-DPM: 100%|██████████| 1050/1050 [03:30<00:00,  4.99it/s]



Сохранены три видео:
  MOT17/MOT17/train/MOT17-04-DPM/output_yolov10.mp4
  MOT17/MOT17/train/MOT17-04-DPM/output_yolov11.mp4
  MOT17/MOT17/train/MOT17-04-DPM/output_ssd.mp4

=== МЕТРИКИ ===
YOLOv10: {'precision': 0.9308847134359362, 'recall': 0.20730912378829616, 'f1': 0.33910022700694775}
YOLOv11: {'precision': 0.9362198599948146, 'recall': 0.30371974683011965, 'f1': 0.4586488846550766}
SSD:     {'precision': 0.8483718617946806, 'recall': 0.14353302352965916, 'f1': 0.245526320522274}

=== Обрабатываем последовательность: MOT17-05-DPM ===
Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...


...


Сохранены три видео:
  MOT17/MOT17/train/MOT17-10-DPM/output_yolov10.mp4
  MOT17/MOT17/train/MOT17-10-DPM/output_yolov11.mp4
  MOT17/MOT17/train/MOT17-10-DPM/output_ssd.mp4

=== МЕТРИКИ ===
YOLOv10: {'precision': 0.867871259175607, 'recall': 0.3591401199470364, 'f1': 0.5080431908329661}
YOLOv11: {'precision': 0.8181818181818182, 'recall': 0.40727470986836983, 'f1': 0.5438377535101404}
SSD:     {'precision': 0.7541387024608501, 'recall': 0.2625593893605421, 'f1': 0.3895083482581316}

=== Обрабатываем последовательность: MOT17-11-DPM ===
Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...


Обработка MOT17-11-DPM: 100%|██████████| 900/900 [03:05<00:00,  4.85it/s]



Сохранены три видео:
  MOT17/MOT17/train/MOT17-11-DPM/output_yolov10.mp4
  MOT17/MOT17/train/MOT17-11-DPM/output_yolov11.mp4
  MOT17/MOT17/train/MOT17-11-DPM/output_ssd.mp4

=== МЕТРИКИ ===
YOLOv10: {'precision': 0.903842954583264, 'recall': 0.5757736328952946, 'f1': 0.7034375606913964}
YOLOv11: {'precision': 0.8698207021244538, 'recall': 0.6118058499364137, 'f1': 0.7183475393517078}
SSD:     {'precision': 0.8831644751023927, 'recall': 0.43418821534548535, 'f1': 0.5821669626998224}

=== Обрабатываем последовательность: MOT17-13-DPM ===
Создаём 3 видео (YOLOv10 / YOLOv11 / SSD)...


Обработка MOT17-13-DPM: 100%|██████████| 750/750 [02:25<00:00,  5.15it/s]



Сохранены три видео:
  MOT17/MOT17/train/MOT17-13-DPM/output_yolov10.mp4
  MOT17/MOT17/train/MOT17-13-DPM/output_yolov11.mp4
  MOT17/MOT17/train/MOT17-13-DPM/output_ssd.mp4

=== МЕТРИКИ ===
YOLOv10: {'precision': 0.8936800526662277, 'recall': 0.23320735268854148, 'f1': 0.3698910081743869}
YOLOv11: {'precision': 0.8821476510067114, 'recall': 0.282253908263185, 'f1': 0.4276696817856446}
SSD:     {'precision': 0.6820877817319099, 'recall': 0.1481704174540457, 'f1': 0.24345494319384658}


**Общие наблюдения:**

* **YOLOv11** в большинстве случаев показывает **наилучший баланс между точностью (Precision) и полнотой (Recall)**, что отражается в более высоких значениях F1-меры.
* **YOLOv10** демонстрирует **высокую точность (Precision)**, но часто **уступает YOLOv11 в полноте (Recall)**, что говорит о том, что он может пропускать часть объектов.
* **SSD** обычно показывает **более низкие значения Recall** по сравнению с YOLO моделями, что указывает на большее количество пропущенных объектов. Его точность (Precision) варьируется, иногда приближаясь к YOLO, но часто остаётся ниже.

* **Точность (Precision):** Обе YOLO модели демонстрируют высокую точность, с небольшим преимуществом у YOLOv10. Это означает, что когда они детектируют объект, вероятность того, что это действительно человек, высока. SSD показывает несколько более низкую точность.
* **Полнота (Recall):** YOLOv11 обычно имеет более высокую полноту, чем YOLOv10 и SSD. Это означает, что YOLOv11 лучше справляется с обнаружением всех присутствующих людей в кадре. SSD показывает наименьшую полноту, что говорит о большем количестве пропущенных обнаружений.
* **F1-мера:**  F1-мера, которая является гармоническим средним между Precision и Recall, показывает, что **YOLOv11 в среднем обеспечивает лучший баланс** между этими двумя метриками. YOLOv10 занимает второе место, а SSD показывает наименьшее значение F1.

**Различия в результатах на разных видеопоследовательностях:**

Наблюдается **зависимость производительности моделей от конкретной видеопоследовательности**. Например, на последовательности `MOT17-05-DPM` все три модели показали относительно высокие значения Recall и F1, в то время как на других последовательностях, таких как `MOT17-02-DPM` и `MOT17-04-DPM`, Recall был значительно ниже. Это может быть связано с различными условиями съемки, плотностью людей, освещением и т.д.

**Качество детектирования на видео:**

На основе визуального анализа созданных видеороликов можно отметить следующее:

* **YOLO модели** в целом **более уверенно детектируют людей**, выделяя их более точно и с меньшим количеством ложных срабатываний, особенно при хорошем освещении и четкой видимости.
* **SSD** иногда **пропускает объекты** или **выделяет их менее точно**, особенно в случаях, когда люди находятся на некотором расстоянии или частично перекрыты другими объектами.

**Выводы:**

* **YOLOv11 представляется наиболее сбалансированной моделью** для задачи детекции людей в видеопотоке, обеспечивая хорошее сочетание точности и полноты.
* **YOLOv10** может быть предпочтительнее в сценариях, где **важнее минимизировать ложные срабатывания**, даже ценой пропуска некоторых объектов.
* **SSD** в текущей конфигурации **уступает YOLO моделям по качеству детекции** на данном наборе данных, особенно в плане полноты обнаружения.
